### 数据清理

> 这个笔记本主要是检查数据的可用性, 将原始数据转换为可用的数据集, 并将其保存到文件中, 以便以后使用.

#### 检查各个数据来源的国家列表

In [ ]:
import pandas as pd
import logging
import numpy as np

In [ ]:
'''
全局变量
'''
LOGGER_LEVEL = logging.DEBUG

#### 设置日志

In [ ]:
'''
配置日志记录
'''


# 初始化日志记录器
def setup_logger(logger_name=None):
    # 创建一个名为 'console_logger' 的日志记录器，用于输出调试信息到控制台
    console_logger = logging.getLogger(logger_name)
    console_logger.setLevel(logging.DEBUG)  # 设置日志级别为 DEBUG，以便输出所有级别的日志消息

    # 删除现有的处理程序
    for handler in console_logger.handlers[:]:
        console_logger.removeHandler(handler)

    console_handler = logging.StreamHandler()  # 创建一个输出到控制台的处理程序
    console_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')  # 定义日志消息的格式
    console_handler.setFormatter(console_formatter)  # 为处理程序设置格式

    console_logger.addHandler(console_handler)  # 将处理程序添加到日志记录器

    # 创建一个名为 'file_logger' 的日志记录器，用于将重要的日志消息写入文件
    file_logger = logging.getLogger(logger_name)
    file_logger.setLevel(logging.INFO)  # 设置日志级别为 INFO，以便只输出 INFO 级别及以上的日志消息

    # 删除现有的处理程序
    for handler in file_logger.handlers[:]:
        file_logger.removeHandler(handler)
    file_handler = logging.FileHandler('log/important_data.log')  # 创建一个输出到文件的处理程序
    file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')  # 定义日志消息的格式
    file_handler.setFormatter(file_formatter)  # 为处理程序设置格式

    file_logger.addHandler(file_handler)  # 将处理程序添加到日志记录器

    return console_logger, file_logger


console_logger, file_logger = setup_logger('data_cleaning')  # 设置日志系统


#### tools functions

In [ ]:
# 将缺失值替换为0
def replace_nan_with_zero(df, column_name):
    # 步骤1: 找到所有独特的国家名称
    unique_countries = df['country_name'].unique()

    # 步骤2: 为每个国家创建一个包含所有年份的新数据帧
    all_frames = []  # 用于存储每个国家的新数据帧

    for country in unique_countries:
        # 获取该国家的现有数据
        country_data = df[df['country_name'] == country]

        # 创建一个包含所有年份的新数据帧
        all_years = pd.DataFrame({'year': np.arange(2006, 2022)})  # 假设所有国家的年份范围都是2006到2021
        all_years['country_name'] = country  # 设置国家名称
        all_years = all_years.merge(country_data, on=['country_name', 'year'], how='left')  # 合并现有数据
        all_years[column_name] = all_years[column_name].fillna(0)  # 将缺失的fsi_index值设置为0

        all_frames.append(all_years)

    # 步骤3: 将这些新数据帧合并为一个大的数据帧
    final_df = pd.concat(all_frames, ignore_index=True)

    return final_df

#### 1.fsi_index.csv

In [ ]:
df = pd.read_csv('data/fsi_index.csv')
# 重命名列名
df.columns = ['country_name', 'year', 'fsi_index']
# 修改值的类型
df = df.convert_dtypes()

# 筛选2006-2021年的数据, 并按照国家名称和年份排序
df = df[(df['year'] >= 2006) & (df['year'] <= 2021)]
# 删除字符串两端的空格
df['country_name'] = df['country_name'].str.strip()

df.sort_values(by=['country_name', 'year'], inplace=True)
# 填补缺失值
df = replace_nan_with_zero(df, 'fsi_index')

# 导入到cleaned_data/fsi_index.csv文件中
df.to_csv('cleaned_data/fsi_index.csv', index=False)

##### 1.1 删除数据严重缺失的国家并填充少量缺失的数据

In [114]:
df = pd.read_csv('cleaned_data/result.csv')

country_list = df['受援对象国'].unique()
#将0.0替换为缺失值
df['对象国安全系数指标'] = df['对象国安全系数指标'].replace(0.0, np.nan)
df['HDI'] = df['HDI'].replace(0.0, np.nan)

for country in country_list:
    country_df = df[df['受援对象国'] == country]
    # 如果出现大于5个缺失值, 则删除该国家的数据
    if country_df['对象国安全系数指标'].isnull().sum() > 5:
        df = df[df['受援对象国'] != country]
    elif country_df['HDI'].isnull().sum() > 5:
        df = df[df['受援对象国'] != country]

# 重索引
df.reset_index(drop=True, inplace=True)
# 填补缺失值
zero_df = df[df['对象国安全系数指标'].isnull()]

for i in zero_df.index:
    if df.loc[i, '年度'] == 2006:
        df.loc[i, '对象国安全系数指标'] = df.loc[i + 1, '对象国安全系数指标']
        df.loc[i, 'HDI'] = df.loc[i + 1, 'HDI']
    elif df.loc[i, '年度'] == 2021:
        df.loc[i, '对象国安全系数指标'] = df.loc[i - 1, '对象国安全系数指标']
        df.loc[i, 'HDI'] = df.loc[i - 1, 'HDI']
    else:
        df.loc[i, '对象国安全系数指标'] = (df.loc[i - 1, '对象国安全系数指标'] + df.loc[
            i + 1, '对象国安全系数指标']) / 2
        df.loc[i, 'HDI'] = (df.loc[i - 1, 'HDI'] + df.loc[i + 1, 'HDI']) / 2

# 年度,受援对象国,受援对象国代码,援助类别,经济援助金额(美元),军事援助金额(美元),总援助金额(美元),SDG,HDI,对象国经济发展指标,对象国安全系数指标,对象国发展-安全综合指数
# 7. 计算每个国家每年的经济发展指标(归一化的SDG+HDI/2)
# 计算sdg归一化指标
max_value = df['SDG'].max()
min_value = df['SDG'].min()

# 手动补充缺失值
null_df = df[df['HDI'].isnull()]
for i in null_df.index:
    df.loc[i, 'HDI'] = 0.581
# todo 如果其中一个指标缺失，如何处理？
df['对象国经济发展指标'] = ((df['SDG'] - min_value) / (max_value - min_value) + df['HDI']) / 2

# 8.计算每个国家每年的发展-安全综合指数(归一化的FSI+经济发展指标/2)
max_value = df['对象国安全系数指标'].max()
min_value = df['对象国安全系数指标'].min()

df['对象国发展-安全综合指数'] = (1 - (df['对象国安全系数指标'] - min_value) / (max_value - min_value)
                                 + df['对象国经济发展指标']) / 2

df.to_csv('cleaned_data/test_all_chinese.csv', index=False)


#### 2.hdi_index.csv

In [95]:
df = pd.read_csv('data/hdi_index.csv')

# 删除不需要的列hdicode,region,hdi_rank_2021
df.drop(['hdicode', 'region', 'hdi_rank_2021'], axis=1, inplace=True)
# 修改列名
df.rename(columns={'iso3': 'country_code', 'country': 'country_name'}, inplace=True)
# 将缺失值替换为0
df.fillna(0.0, inplace=True)
# 将各个年份的index值由多个列转换为一列, 原始列名为hdi_2006,hdi_2007,hdi_2008,hdi_2009,hdi_2010,
# hdi_2011,hdi_2012,hdi_2013,hdi_2014,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019,hdi_2020,hdi_2021
# 需要转换为year一个列
df = pd.melt(df, id_vars=['country_code', 'country_name'], var_name='year', value_name='hdi_index')
# 排序
df.sort_values(by=['country_name', 'year'], inplace=True)
# 将year列中的hdi_删除, 并将其余的值转换为int类型
df['year'] = df['year'].str.replace('hdi_', '').astype(int)
# 修改值的类型
df = df.convert_dtypes()

# 导入到cleaned_data/hdi_index.csv文件中
df.to_csv('cleaned_data/hdi_index.csv', index=False)

Empty DataFrame
Columns: [country_code, country_name, year, hdi_index]
Index: []


#### 3.SDG_all.csv

In [ ]:
df = pd.read_csv('data/SDG_all.csv')

# 保留'Country Code ISO3', 'Country', 'year', 'SDG Index Score'
# 并将他们重命名为'country_code', 'country_name', 'year', 'sdg_index'
df = df[['Country Code ISO3', 'Country', 'year', 'SDG Index Score']]
df.columns = ['country_code', 'country_name', 'year', 'sdg_index']
# 修改值的类型
df = df.convert_dtypes()

# 筛选2006-2021年的数据, 并按照国家名称和年份排序
df = df[(df['year'] >= 2006) & (df['year'] <= 2021)]
df.sort_values(by=['country_name', 'year'], inplace=True)

# 导出到cleaned_data/sdg_index.csv文件中
df.to_csv('cleaned_data/sdg_index.csv', index=False)

#### 4. amount.csv

In [ ]:
df = pd.read_csv('data/us_foreign_aid_part.csv')

# 删除'1976tq'对应的行
df = df[df['Fiscal Year'] != '1976tq']

# 修改列名
df.columns = ['country_code', 'country_name', 'year', 'assistance_type', 'amount']

# 将'year'列的值转换为int类型
df['year'] = df['year'].astype(int)
# 修改值的类型
df = df.convert_dtypes()

# 筛选2006-2021年的数据, 并按照国家名称和年份排序
df = df[(df['year'] >= 2006) & (df['year'] <= 2021)]
df.sort_values(by=['country_name', 'year'], inplace=True)

# 将同一国家同一年份同一类型的数据合并为一行
# 使用 groupby 方法按 country_code, country_name, year 和 assistance_type 进行分组
grouped = df.groupby(['country_code', 'country_name', 'year', 'assistance_type'])

# 对每个组的 amount 列求和
summed = grouped['amount'].sum().reset_index()

summed.sort_values(by=['country_name', 'year'], inplace=True)
# # 将缺失值替换为0
# summed = replace_nan_with_zero(summed, 'amount')
# 导出到cleaned_data/amount.csv文件中
summed.to_csv('cleaned_data/test_amount.csv', index=False)


#### 筛选出一个每个指标都有的国家列表

In [101]:
'''
每个数据集都有一个国家列表, 但是每个数据集的国家列表都不同.
因此, 我们需要找到所有数据集中的国家列表, 然后找到所有数据集中的国家列表的交集.
'''

# 数据集列表
data_sets = [
    'cleaned_data/fsi_index.csv',
    'cleaned_data/hdi_index.csv',
    'cleaned_data/sdg_index.csv',
    'cleaned_data/amount.csv'
]

'''
读取每个数据集中的国家代码,如果有重复的国家,则只保留一个,
其中fsi数据集没有国家代码, 只有国家名称, 因此需要将国家名称转换为国家代码
'''

# 从上面的输出中可以看出, fsi_index.csv没有country_code列, 只有country_name列
# 我们需要先将country_name转换为country_code, 然后再进行合并

# 读取fsi_index.csv
df = pd.read_csv('cleaned_data/fsi_index.csv')
# 读取country.csv
country_df = pd.read_csv('cleaned_data/country.csv')
# 将fsi_index.csv中的country_name列与country.csv中的country_name列进行合并
df = df.merge(country_df, on='country_name', how='left')
# year country_name  fsi_index country_code_x country_code_y
# 删除country_code_x列, 并将country_code_y列重命名为country_code
df.drop(['country_code_x'], axis=1, inplace=True)
df.rename(columns={'country_code_y': 'country_code'}, inplace=True)

# 删除country_code为空值的行
df = df[df['country_code'].notnull()]

df.to_csv('cleaned_data/fsi_index.csv', index=False)

# 求不同数据集中的国家列表的交集
# 读取第一个数据集
df = pd.read_csv(data_sets[0])
# 读取第二个数据集
df2 = pd.read_csv(data_sets[1])
# 读取第三个数据集
df3 = pd.read_csv(data_sets[2])
# 读取第四个数据集
df4 = pd.read_csv(data_sets[3])

# 将第一个数据集中的国家列表与第二个数据集中的国家列表进行合并
df = df.merge(df2, on=['country_code', 'country_name', 'year'], how='inner')
# 将第一个数据集中的国家列表与第三个数据集中的国家列表进行合并
df = df.merge(df3, on=['country_code', 'country_name', 'year'], how='inner')
# 将第一个数据集中的国家列表与第四个数据集中的国家列表进行合并
df = df.merge(df4, on=['country_code', 'country_name', 'year'], how='inner')

# year,country_name,fsi_index,country_code,hdi_index,sdg_index,assistance_type,amount
# 改变列的顺序
df = df[['year', 'country_name', 'country_code', 'fsi_index', 'hdi_index', 'sdg_index', 'assistance_type', 'amount']]

# 将结果保存到cleaned_data/country_code.csv文件中
df.to_csv('cleaned_data/test_all.csv', index=False)

#### Test

In [ ]:
import pandas as pd

# 创建一个宽格式的 DataFrame
df = pd.DataFrame({'A': ['a1', 'a2', 'a3'],
                   'B': ['b1', 'b2', 'b3'],
                   'C': ['c1', 'c2', 'c3']})

# 融化 DataFrame，保留 'A' 列，融化 'B' 和 'C' 列
melted_df = pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

# 输出结果 DataFrame
print(melted_df)

In [ ]:
df = pd.read_csv('data/country.csv')
print(df.columns)
df = df.convert_dtypes()
print(df.dtypes)
print(df.head())
# 去除字符串两端的空格
df['country_name'] = df['country_name'].str.strip()
df['country_code'] = df['country_code'].str.strip()
df.to_csv('cleaned_data/country.csv', index=False)


In [ ]:
#读取amount.csv, 查找缺失的国家代码
amount_df = pd.read_csv('cleaned_data/test_amount.csv')
amount_df = amount_df[amount_df['country_name'].isin(null_country_name)]
amount_df = amount_df[['country_code', 'country_name']]
amount_df.drop_duplicates(inplace=True)
print(amount_df)
# 将amount_df中的country_code列与country.csv中的country_name列进行合并(求并集)
country_df = pd.concat([country_df, amount_df], ignore_index=True)
country_df.drop_duplicates(inplace=True)
country_df.to_csv('cleaned_data/country.csv', index=False)

In [ ]:
'''
将这些添加进country.csv
'Brunei Darussalam', BRN

 'Congo Democratic Republic', COD
 'Guinea Bissau', GNB

 'Macedonia', MKD

'Micronesia', FSM
 'North Korea', PRK

 'Palestine', PSE

'Slovakia', SVK

'South Korea', KOR
'''

df = pd.read_csv('cleaned_data/country.csv')

# 读取json字符串
import json

with open('data/list.json', 'r') as f:
    country_code = json.load(f)
    print(country_code)

# 将json字符串转换为DataFrame
country_code_df = pd.DataFrame(country_code)

# 将country_code_df中的country_code列与country.csv中的country_name列进行合并(求并集)
country_df = pd.concat([df, country_code_df], ignore_index=True)
country_df.drop_duplicates(inplace=True)
# 排序
country_df.sort_values(by=['country_name'], inplace=True)
country_df.to_csv('cleaned_data/country.csv', index=False)

In [ ]:
df = pd.read_csv('cleaned_data/all.csv')
